In [9]:
num_epochs = 10
batch_size = 32
best_accuracy = 0

In [10]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [11]:
class MyCNN(nn.Module):
    def __init__(self, num_layers = 10, input_features=34, input_channels=29, output_features=34):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()

        self.conv_layers.append(nn.Conv1d(in_channels=input_channels, out_channels=512, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(
                nn.Sequential(
                    nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                    nn.BatchNorm1d(512),
                    nn.LeakyReLU(0.01)
                    )
            )
        
        self.conv_layers.append(nn.Conv1d(in_channels=512, out_channels=1, kernel_size=1, padding=0))
        
        self.activation = nn.LeakyReLU(negative_slope=0.01)

        self.fc = nn.Linear(input_features, output_features)
    def forward(self, x):
        x = x.squeeze(1)
        x = x.permute(0, 2, 1)

        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = self.activation(conv(x))
            else:
                x = conv(x)
                
        x = x.squeeze(1)
        x = self.fc(x)
        print("x.shape:",x.shape)
        
        return x

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [13]:
models_path = [
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_37.5.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250927_082928.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250927_052753.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250927_022657.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250926_232602.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250926_202430.pth",
    "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_20250926_172152.pth"
]

model_list = [torch.load(p, map_location=device).to(device).eval() for p in models_path]
num_models = len(model_list)

loss_criterion  = nn.CrossEntropyLoss()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\timy3\AppData\Local\Temp\ipykernel_5416\3970616742.py:2: SyntaxWarning: invalid escape sequence '\d'
  "E:\專題\discard_model\DiscardModel_512_Bat_Res\CNN_discard_model_37.5.pth",
C:\Users\timy3\AppData\Local\Temp\ipykernel_5416\3970616742.py:3: SyntaxWarning: invalid escape sequence '\d'
  "

In [14]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'discard']  # 排除 'discard' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        if 'discard' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Discard' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values.astype("float32")
        train_data = train_data.reshape(34, 29)
        
        value_data = df['discard'].iloc[sample_idx]  # 提取 'Discard' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.long)
        
    def __len__(self):
        return self.num_samples

In [15]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2022/DiscardData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
correct_list = [0] * num_models
loss_list = [0.0] * num_models
total = 0

with torch.no_grad():
    for train_data, value_data in test_dataloader:
        torch.autograd.set_detect_anomaly(True)
        channel1_tensor = train_data.unsqueeze(1)
        channel1_tensor = channel1_tensor.to(device)
        value_data = value_data.to(device).long()
        batch_size = value_data.size(0)
        total += batch_size

        rows = []

        for i, model in enumerate(model_list):
        
            output = model(channel1_tensor).float()
            probabilities = F.softmax(output, dim=1)
            predicted_labels = torch.argmax(probabilities, dim=1)
            correct_predictions = (predicted_labels == value_data).sum().item()
            correct_list[i] += correct_predictions

            loss = loss_criterion(output, value_data)
            loss_list[i] += loss.item() * batch_size

            accuracy = (correct_predictions / batch_size) * 100
            total_accuracy = (correct_list[i] / total) * 100

            rows.append({
                "Model": i,
                "Cur Acc (%)": f"{accuracy:.2f}",
                "Total Acc (%)": f"{total_accuracy:.2f}",
                "Loss": f"{loss.item():.4f}",
            })
        
        clear_output(wait=True)
        df = pd.DataFrame(rows)
        display(df)

,Model,Cur Acc (%),Total Acc (%),Loss
0,0,3.12,4.77,3.5113
1,1,9.38,5.26,3.4922
2,2,9.38,5.26,3.4804
3,3,9.38,5.26,3.4682
4,4,9.38,5.26,3.4739
5,5,9.38,5.26,3.4801
6,6,9.38,5.26,3.4841
